<a href="https://colab.research.google.com/github/frankwillard/StatGPT/blob/main/GPT_2_Champion_Weighted_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00


## Load In Libraries, Data

In [ ]:
import pandas as pd

import torch
from torch.utils.data import Dataset

from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2ForSequenceClassification, TrainingArguments, Trainer

In [ ]:
team_data = pd.read_csv('nba_team_advanced_data (1).csv')

In [ ]:
team_data.head()

,Year,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,DRB%,DFT/FGA,Arena,Attend.,Playoffs,W/L%,Losing_season,Champion,won_last,won_last_3
0,1990.0,Atlanta Hawks,28.6,41.0,41.0,44.0,38.0,1.02,-0.39,0.64,...,65.5,0.254,Omni Coliseum,573711.0,N,0.500000,N,N,N,N
1,1990.0,Boston Celtics,30.2,52.0,30.0,51.0,31.0,3.99,-0.76,3.23,...,72.1,0.223,Boston Garden,611537.0,Y,0.634146,N,N,N,N
2,1990.0,Charlotte Hornets,25.6,19.0,63.0,21.0,61.0,-7.82,0.81,-7.00,...,67.5,0.269,Charlotte Coliseum,979941.0,N,0.231707,Y,N,N,N
3,1990.0,Chicago Bulls,26.1,55.0,27.0,50.0,32.0,3.26,-0.51,2.74,...,68.1,0.262,Chicago Stadium,752564.0,Y,0.670732,N,N,N,N
4,1990.0,Cleveland Cavaliers,26.1,42.0,40.0,40.0,42.0,-0.30,-0.31,-0.62,...,67.7,0.202,Coliseum at Richfield,695710.0,Y,0.512195,N,N,N,N


In [ ]:
team_data.columns

Index(['Year', 'Team', 'Age', 'W', 'L', 'PW', 'PL', 'MOV', 'SOS', 'SRS',
       'ORtg', 'DRtg', 'NRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'OeFG%', 'OTOV%',
       'ORB%', 'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'Arena',
       'Attend.', 'Playoffs', 'W/L%', 'Losing_season', 'Champion', 'won_last',
       'won_last_3'],
      dtype='object')

In [ ]:
# Miami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship.
df = team_data[["Year", "Team", "Age" , "MOV" , "NRtg" ,"TS%", "W/L%", 'won_last']]
# df.rename(columns={'PTS_per_game': 'Points', 'TRB_per_game': 'Rebounds', 'AST_per_game': 'Assists', 'STL_per_game': 'Steals', 'BLK_per_game': 'Blocks', "VORP_advanced" : "vorp"}, inplace=True)


# Convert all columns to integer type
df = df.astype(float, errors='ignore')

# cond1 = (df["Points"] < 99) & (df["Rebounds"] < 99) & (df["Assists"] < 99) & (df["Steals"] < 99) & (df["Blocks"] < 99)
# cond2 = (df["Points"] > 0) & (df["Rebounds"] > 0) & (df["Assists"] > 0) & (df["Steals"] > 0) & (df["Blocks"] > 0)

# df = df[cond1]
# df = df[cond2]

df.to_csv('teams_stats.csv', index=False)

#player_data = player_data[cond]


In [ ]:
# all_nba = player_data[["Player", "All_NBA"]]
# all_nba.rename(columns={"All_NBA": 'allnba'}, inplace=True)

# all_nba =  all_nba[all_nba["Player"].isin(df["Player"])]
# all_nba.to_csv('all_nba.csv', index=False)

In [ ]:
# vorp = player_data[["Player", "VORP_advanced"]]
# vorp.rename(columns={"VORP_advanced": 'vorp'}, inplace=True)
# vorp =  vorp[vorp["Player"].isin(df["Player"])]
# vorp.head()
# vorp.to_csv('vorp.csv', index=False)


# vorp['vorp'] = vorp['vorp'].apply(lambda x: 0 if x <= 0 else 1)
# vorp.to_csv('vorp_binary.csv', index=False)

In [ ]:
champion = team_data[["Year", "Team", 'Champion']]

In [ ]:
len(champion)

963

In [ ]:
champion['Champion'] = champion['Champion'].apply(lambda x: 0 if x == 'N' else 1)

<ipython-input-48-fcf44467c76d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  champion['Champion'] = champion['Champion'].apply(lambda x: 0 if x == 'N' else 1)


In [ ]:
champion['Champion'].value_counts()

0    930
1     33
Name: Champion, dtype: int64

In [ ]:
champion.to_csv('champion_binary.csv', index=False)

## Define Train/Test Split

In [ ]:
team_stats = pd.read_csv('teams_stats.csv')
# vorp = pd.read_csv('vorp_binary.csv')
champion = pd.read_csv('champion_binary.csv')

In [ ]:
# prompt: players_stats and vorp have the same players, randomly split the dataset into train and test sets where player_stats are the input and vorp is the label. Players in both sets should be the same. Set the indexing of the pd df to start from 0

# from sklearn.model_selection import train_test_split
# train_data, test_data, train_labels, test_labels = train_test_split(team_stats, champion, test_size=0.25, random_state=42)

# train_data.index = range(len(train_data))
# train_labels.index = range(len(train_labels))
# test_data.index = range(len(test_data))
# test_labels.index = range(len(test_labels))


In [ ]:
train_data = team_stats[team_stats['Year'] < 2013]
test_data = team_stats[team_stats['Year'] > 2014]
train_labels = champion[champion['Year'] < 2013]
test_labels = champion[champion['Year'] > 2014]

In [ ]:
train_data.index = range(len(train_data))
train_labels.index = range(len(train_labels))
test_data.index = range(len(test_data))
test_labels.index = range(len(test_labels))

## Convert Tabular Data to Text Input

In [ ]:
train_data.iloc[0]['won_last']

'N'

In [ ]:
training_inputs = []

# Miami Heat in 2013 has average Age: 30.3, MOV: 7.87, NRtg: 8.6, TS: 0.588, Win Percentage: 0.805, and won last year's championship.
for index, row in train_data.iterrows():
  # Will include more in future
  won_last = 'did not win' if row['won_last'] == 'N' else 'won'
  train_input = f"{row['Team']} in {row['Year']} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {row['W/L%']} and {won_last} last year's championship"
  training_inputs.append(train_input)

training_labels = []
for _,row in train_labels.iterrows():
  # Will include more in future
  train_label = f"{row['Champion']}"
  training_labels.append(int(train_label))

In [ ]:
# Initialize empty lists for test inputs and labels
testing_inputs = []
testing_labels = []

# Loop through test_data for inputs
for index, row in test_data.iterrows():
    # Will include more in the future
    won_last = 'did not win' if row['won_last'] == 'N' else 'won'
    test_input = f"{row['Team']} in {row['Year']} has average Age: {row['Age']}, MOV: {row['MOV']}, NRtg: {row['NRtg']}, TS: {row['TS%']}, Win Percentage: {row['W/L%']} and {won_last} last year's championship"
    testing_inputs.append(test_input)

# Loop through test_labels for labels
for _, row in test_labels.iterrows():
    # Will include more in the future
    test_label = f"{row['Champion']}"
    testing_labels.append(int(test_label))


In [ ]:
len(training_inputs)

663

# Define Model and Start Training

In [ ]:
model_name = "gpt2-large"  # or other sizes like "gpt2-medium", "gpt2-large", etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model.cuda()



tokenized_training_data = tokenizer(training_inputs, padding=True)
tokenized_testing_data = tokenizer(testing_inputs, padding=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class Dataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.tokenized_data = tokenized_data
        self.labels = labels

    def __len__(self):
        return len(self.tokenized_data["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.tokenized_data.items()}

        item["labels"] = torch.tensor(self.labels[idx])

        return item

In [ ]:
# Define training dataset
training_dataset = Dataset(tokenized_training_data, training_labels)
testing_dataset = Dataset(tokenized_testing_data, testing_labels)

In [ ]:
len(training_dataset)

663

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch.nn as nn

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 29.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(p):

    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "recall": recall, "f1": f1}

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    seed=0
)


# Initialize the Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=testing_dataset,
    compute_metrics=compute_metrics
)

# Start fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall,F1
1,No log,0.729313,0.941667,0.125000,0.125000
2,No log,0.743768,0.941667,0.125000,0.125000
3,No log,0.623437,0.758333,0.750000,0.171429
4,No log,0.679298,0.937500,0.125000,0.117647
5,No log,0.641456,0.937500,0.250000,0.210526


TrainOutput(global_step=55, training_loss=0.617727314342152, metrics={'train_runtime': 42.4065, 'train_samples_per_second': 78.172, 'train_steps_per_second': 1.297, 'total_flos': 99815869071360.0, 'train_loss': 0.617727314342152, 'epoch': 5.0})

In [ ]:
# count = 0
# total = 0
# batch_size = 64

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)


# model = model.cuda()
# for i in range(0, len(testing_inputs), batch_size):
#     sentences = testing_inputs[i:i+batch_size]
#     labels = torch.tensor(testing_labels[i:i+batch_size])
#     t = tokenizer(sentences, return_tensors="pt", padding = True)
#     t = {key: val.cuda() for key, val in t.items()}

#     pred = model.forward(**t).logits.argmax(-1).detach().cpu()


#     count += (pred == labels).sum()


# print(count)
# print(len(testing_inputs))

In [ ]:
count = 0
total = 0
batch_size = 64

TP, FP, TN, FN = 0, 0, 0, 0

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model = model.cuda()

for i in range(0, len(testing_inputs), batch_size):
    sentences = testing_inputs[i:i + batch_size]
    labels = torch.tensor(testing_labels[i:i + batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding=True)
    t = {key: val.cuda() for key, val in t.items()}

    pred = model.forward(**t).logits.argmax(-1).detach().cpu()

    TP += ((pred == 1) & (labels == 1)).sum().item()
    TN += ((pred == 0) & (labels == 0)).sum().item()
    FP += ((pred == 1) & (labels == 0)).sum().item()
    FN += ((pred == 0) & (labels == 1)).sum().item()

# Calculating the metrics
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(TP, TN, FP, FN)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

2 223 9 6
Accuracy: 0.9375
Precision: 0.18181818181818182
Recall: 0.25
F1 Score: 0.2105263157894737


In [ ]:
import torch.nn.functional as F

In [ ]:
count = 0
total = 0
batch_size = 30

TP, FP, TN, FN = 0, 0, 0, 0

# model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2, pad_token_id=tokenizer.pad_token_id)
model = model.cuda()

for i in range(0, len(testing_inputs), batch_size):
    sentences = testing_inputs[i:i + batch_size]
    labels = torch.tensor(testing_labels[i:i + batch_size])
    t = tokenizer(sentences, return_tensors="pt", padding=True)
    t = {key: val.cuda() for key, val in t.items()}

    champion_preds = model.forward(**t).logits.detach().cpu()[:,1]
    champion_probs = F.softmax(champion_preds, dim=0)

    predicted_champion = torch.topk(champion_preds, 3).indices
    predicted_probs = champion_probs[predicted_champion]

    print(testing_inputs[i + predicted_champion[0]])
    print(testing_inputs[i + predicted_champion[1]])
    print(testing_inputs[i + predicted_champion[2]])
    print(predicted_probs)
    print()

San Antonio Spurs in 2015.0 has average Age: 29.8, MOV: 6.2, NRtg: 6.5, TS: 0.555, Win Percentage: 0.670731707317073 and won last year's championship
Memphis Grizzlies in 2015.0 has average Age: 29.6, MOV: 3.24, NRtg: 3.5, TS: 0.531, Win Percentage: 0.670731707317073 and did not win last year's championship
Golden State Warriors in 2015.0 has average Age: 26.6, MOV: 10.1, NRtg: 10.2, TS: 0.571, Win Percentage: 0.817073170731707 and did not win last year's championship
tensor([0.8583, 0.0108, 0.0089])

Golden State Warriors in 2016.0 has average Age: 27.4, MOV: 10.76, NRtg: 10.7, TS: 0.593, Win Percentage: 0.890243902439024 and won last year's championship
San Antonio Spurs in 2016.0 has average Age: 30.3, MOV: 10.63, NRtg: 11.3, TS: 0.564, Win Percentage: 0.817073170731707 and did not win last year's championship
Los Angeles Lakers in 2016.0 has average Age: 26.5, MOV: -9.56, NRtg: -10.0, TS: 0.509, Win Percentage: 0.207317073170732 and did not win last year's championship
tensor([0.43